## 20단계: 연산자 오버로드(1)

> 앞으로는 `Variable` 인스턴스를 `ndarray` 인스턴스 처럼 보이게 만드는 것이 목표입니다. \
이를 위해 `+`, `*` 연산자 등을 지원하도록 `Variable`을 확장하겠습니다. \
이번 단계에서는 곱셈을 지원하는 `Mul` 클래스를 구현하고, 연산자를 오버로드하여 사용성을 개선해보겠습니다.

### 20.1 Mul 클래스 구현

곱셈의 미분은 $y = x_0 \times x_1$일 때 $\frac{\partial y}{\partial x_0} = x_1$, $\frac{\partial y}{\partial x_1} = x_0$이 된다.

<img src="images/그림 20-1.png" width=500/>

위 그림을 참고하여 곱셈을 구현해보자.

In [ ]:
# 필요 모듈 정의

import weakref
import contextlib
from heapq import heappush, heappop
import numpy as np


class Config:
    enable_backprop = True  # True이면 역전파 활성 모드


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


def as_array(x):
    if np.isscalar(x):
        return np.asarray(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        # 역전파 활성 모드 확인
        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])  # Todo: __lt__와 충돌 가능
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.proxy(output) for output in outputs]
        
        return outputs if len(outputs) > 1 else outputs[0]

    def __lt__(self, other):
        return self.generation > other.generation
    
    def forward(self, xs):
        raise NotImplementedError()
    
    def backward(self, gys):
        raise NotImplementedError()


class Variable:
    def __init__(self, data, name=None):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return 'Variable(None)'
        p = str(self.data).replace('\n', '\n' + ' '*9)
        return f'Variable({p})'
    
    @property
    def shape(self):
        return self.data.shape
    
    @property
    def ndim(self):
        return self.data.ndim
    
    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype
    
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
    
    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs_heap = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                heappush(funcs_heap, f)
                seen_set.add(f)
        
        add_func(self.creator)
        
        while funcs_heap:
            f = heappop(funcs_heap)
            gys = [output.grad for output in f.outputs]
            
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
            
            if not retain_grad:
                for y in f.outputs:
                    y.grad = None
    
    def cleargrad(self):
        self.grad = None

In [ ]:
# Mul 구현

class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    return Mul()(x0, x1)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def add(x0, x1):
    return Add()(x0, x1)

def square(x):
    return Square()(x)



a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a, b), c)
y.backward()

print(y)
print(a.grad)
print(b.grad)

Variable(7.0)
2.0
3.0


### 20.2 연산자 오버로드

`__mul__`, `__add__` 등의 특수한 메서드들을 정의하면 `*`, `+` 등의 기호를 사용하여 연산할 수 있다.

In [4]:
# 정석 코드
# class Variable:
#     ...
    
#     def __mul__(self, other):
#         return mul(self, other)


# 다른 방법으로, 아래와 같이 쉽게 추가 가능하다
Variable.__mul__ = mul
Variable.__add__ = add

In [5]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

# y = add(mul(a, b), c)
y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)

Variable(7.0)
2.0
3.0
